### Environment Config

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

### Read Dataset

Data was preprocessed by other script available in: https://github.com/HWatanuki/Projeto_D3TOP/blob/main/Codigos/Data_preprocessing_v0.3.ipynb

In [2]:
file_path = 'https://media.githubusercontent.com/media/HWatanuki/Projeto_D3TOP/main/Datasets/processed/'
file_name = 'dataset_airbnb-processed-spacy_2023-04-13_03-28-09-439.csv'

df = pd.read_csv(file_path + file_name, dtype=str)
print(f'No Rows: {df.shape[0]}')
print(f'No Columns: {df.shape[1]}', end='\n\n')
display(df.head())

No Rows: 561
No Columns: 94



,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/listingsCount,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,primaryHost/totalListingsCount,reviews/0/author/firstName,reviews/0/author/id,reviews/0/createdAt,reviews/0/id,reviews/0/rating,reviews/1/author/firstName,reviews/1/author/id,reviews/1/createdAt,reviews/1/id,reviews/1/rating,reviews/2/author/firstName,reviews/2/author/id,reviews/2/createdAt,reviews/2/id,reviews/2/rating,reviews/3/author/firstName,reviews/3/author/id,reviews/3/createdAt,reviews/3/id,reviews/3/rating,reviews/4/author/firstName,reviews/4/author/id,reviews/4/createdAt,reviews/4/id,reviews/4/rating,reviews/5/author/firstName,reviews/5/author/id,reviews/5/createdAt,reviews/5/id,reviews/5/rating,reviews/6/author/firstName,reviews/6/author/id,reviews/6/createdAt,reviews/6/id,reviews/6/rating,reviews/7/author/firstName,reviews/7/author/id,reviews/7/createdAt,reviews/7/id,reviews/7/rating,reviews/8/author/firstName,reviews/8/author/id,reviews/8/createdAt,reviews/8/id,reviews/8/rating,reviews/9/author/firstName,reviews/9/author/id,reviews/9/createdAt,reviews/9/id,reviews/9/rating,roomType,stars,url,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments
0,"Jersey City, New Jersey, United States",true,false,40.7233,-74.03946,2,NaN,https://a0.muscache.com/im/pictures/miso/Hosting-53775685/original/0812b9a1-2d2e-4725-b51f-881045a939b2.jpeg?aki_policy=large,https://a0.muscache.com/im/pictures/miso/Hosting-53775685/original/0812b9a1-2d2e-4725-b51f-881045a939b2.jpeg?aki_policy=small,99999,"$99,999",USD,false,nightly,82 Reviews,Identity verified,NaN,Joes,false,true,245395267,false,中文 (简体),8,Joined in February 2019,NaN,NaN,Joes,14,Miranda,74889676.0,2022-06-20T19:58:54Z,6.536263641559459e+17,4,Abhimanyu,4800160.0,2022-05-23T18:57:26Z,6.333017075212511e+17,5,Robert,162871850.0,2022-05-06T17:28:10Z,6.209355874005248e+17,5,Yu,303996502.0,2022-01-09T21:55:09Z,5.362712073993113e+17,5,Mitesh,253080011.0,2021-12-26T20:27:02Z,5.260799932553211e+17,4.0,Arusha,103758126.0,2021-12-22T21:29:10Z,5.2321216515417446e+17,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entire rental unit,4.67,https://www.airbnb.com/rooms/53775685,entire cozy unit mins manhattan,hello welcome love see world able help people feel right home,great location nice building love parking include,stay perfect mention listing location min walk newport path right opposite huge grocery store joes super kind early checkin late checkout,excellent place stay central fo travel nyc work travel new jersey stay months apartment place advertise joes excellent host,best airbnb stay get great view outside window comfortable convenient living experience settle great,nice apartment great locality apartment list definitely rate star airbnb pet hair carpet couch month old bit concerned have allergies look clean place joes approachable easy communicate,customer take chance trust listing apartment exceed expectations super clean aesthetically pleasing minimalistic vibe definitely luxury apartment building convenient location right shoprite near waterfront path host responsive questions thanks great stay,NaN,NaN,NaN,NaN
1,"New York, United States",true,false,40.70641,-74.0092,2,NaN,https://a0.muscache.com/im/pictures/e2388507-1f5f-4000-aa1b-d3b2279e682a.jpg?aki_policy=large,https://a0.muscache.com/im/pictures/e2388507-1f5f-4000

### Data preparation

In [3]:
# Join all reviews comments for each announcement record
df_s01 = df.copy()
df_s01['comments'] = df_s01[df_s01.columns[df_s01.columns.str.contains('comments')]].agg(lambda x: '  '.join([str(c) for c in x.values if str(c) != 'nan']), axis=1).T

In [67]:
df_s02 = df_s01[['address', 'numberOfGuests', 'name', 'pricing/rate/amount', 'roomType', 'stars',  'comments']].copy()
df_s02.sample(2)

,address,numberOfGuests,name,pricing/rate/amount,roomType,stars,comments
232,"New York, United States",16,celebrity townhouse manhattan nyc,1250,Entire townhouse,4.98,stay day trip place accommodate easily bedroom modern finish provide nice homey feel neighborhood pleasant evenings easy access essential shops pharmacy food market restaurants deli ect ubers available short wait quick easy rides city best form transport find beds little small comfortable bathrooms nice great water pressure thing issue internet cut bit little bit moderately price townhouse huge occupy main area house house separate service entrance owner owner occupy service area super discrete quiet self check self check socialize owner laundry occupy main areas house feel super safe private times thank david hospitality see ny david great host fast response care lot customers family great time stay david beautiful artistically decorate home love return stay david home time nyc host nice place amazing love highly suggest stay place stay amazing trust disappoint point nice townhouse harlem outstanding place david quick answer questions appreciate fantastic home wonderful host love david house perfect like neighbourhood shops close proximity subway super handy get places easy cab uber home definitely come husband choose invite love ones stay davids property week long stay nyc elopement impressions david townhouse extremely positive david go accommodate communications stellar immediate provide essentials set food water westie truly appreciate special details generosity earn stars space pristine serene comfortable parks boardwalk hudson river stunning nt opportunity check outdoor sport center moments away location perfect althletes love spend time outdoors enjoy moments family highly recommend davids townhouse anyones stay city
351,"New York, United States",6,times square sky palace,621,Entire rental unit,4.98,amazing stay great apartment convenient location midtown views stunning host responsive helpful wouldn hesitate recommend stay wonderful property accomodation perfect base trip adults kids clearly ny busy have sanctuary property important especially stay longer period time like didstav communication assistance way process stay brilliant welcome cityi highly recommend property great host great location great apartment definitely stay perfect family plenty beds space great place stay new york visit request thank good pictures place justice clean amazing views centrally locate host attentive check soon get great time thank wonderful clean apartment perfect family great location nyc dinner shopping sights beat views friendly host fast reliable communication plan visit great weekend place great location walk distance spacious apartment big bedrooms love layout feel safe doorman location plan go sure best hosting experience definitely book future ss posting stay EMOJI perfect location trip things decent walking distance use cane beds comfortable place spacious love desk security fast elevators amazing host responsive accommodate place go favorites list highly recommend


### TF-IDF for get relevant words

In [71]:
max_words = 10000
vectoriser = TfidfVectorizer(max_features=max_words)
tfidf_result = vectoriser.fit_transform(df_s02['comments'])
feature_names = vectoriser.get_feature_names_out()
print(f'Data Transformed.')
print('No. of feature_words: ', len(feature_names))

Data Transformed.
No. of feature_words:  6968


In [80]:
def get_summary(row):
  '''
  Function that iterates through the comment (all reviews together), calculating a score for each sentence by summing the TF-IDF values for each word appearing in the sentence. 
  Normalize the sentence scores by dividing by the number of tokens in the sentence (to avoid bias in favor of longer sentences). 
  Then sort the sentences by their scores, and return the highest-scoring sentences as the summary. 
  The number of sentences returned corresponds to roughly 20% of the overall length of the comment.
  '''
  import math
  
  comment_id = row.name  
  comment_text = row['comments']  
  if not len(comment_text):
    return ''
  tfidf_matrix = vectorizer.fit_transform(comment_text.split('  '))
  summary_parts = []
  for i in range(len(comment_text.split('  '))):
    tfidf_scores = tfidf_matrix[i].toarray().flatten()
    highest_tfidf_index = tfidf_scores.argmax()
    highest_tfidf_value = tfidf_scores[highest_tfidf_index]
    highest_tfidf_feature = vectorizer.get_feature_names_out()[highest_tfidf_index]
    summary_parts.append(highest_tfidf_feature)
  return ' '.join(summary_parts)
  #return summary_parts

  




#### Create a new variable containing more relevant words of reviews as the summary

In [81]:
%%time
df_s02['relevant_words'] = df_s02.apply(lambda x: get_summary(x) , axis=1)
df_s02.sample(3)

CPU times: user 3.39 s, sys: 22.2 ms, total: 3.42 s
Wall time: 3.51 s


,address,numberOfGuests,name,pricing/rate/amount,roomType,stars,comments,relevant_words
129,"New York, United States",6,central park rooftop large luxury rooms,833,Entire rental unit,4.78,wonderful stay thank excellent hospitality EMOJI beautiful house clean modern thank mara alexis hospitalityi recommend sure great trip place great loved have open rooftop kitchen nice didn realize separate live area accept main bedroom guests biggest complaint elevator slow access stairs great host great location clean comfortable space wasn totally private people stay floors lock bedroom access area want great placeto stay location park perfect hosts great definitely travel nyc highly recommend alexis mara friendly helpful host group holiday travel nyc location superb local restaurants shops walk distance easy taxi places go help times need let use steam iron printing event tickets place super spacious plenty natural light ask better accommodation big groups holiday visit nyc great trip ny christmas holidays enjoy time city alexis vrbo property clean location excellent upper east manhattan easy access subway great eat establishments property downside know share spaces laundry unit rent small elevator access space house people elevator couldn use stairway access portion house people rent space levels host mara alexis helpful need delightful overall completely private amazing place better host great location alexis mara great enjoy place nearby think subway restaurant bus stop grocery storeshopping take mins reach moma central park love come new york,emoji alexis hospitalityi great trip area great holiday access completely amazing bus
411,"New York, United States",4,bedroom upper west private entrance,627,Entire rental unit,4.94,quintessential manhattan neighborhood walk distance museums central park great restaurants family fabulous vacation feel comfortable lovely home home spotless wellappointed need days explore city home cozy welcome respite surely return trip host helpful responsive leave great recommendations struggle open lock arrival wasn difficultwe wrong host assist immediately kind word smile truly place jed place perfect family great location easy navigate city space lounge hotel offer jed extremely responsive accommodate great place ideal family wonderful week boys years old central park american museum natural history childrens museum walk distance min restaurants close subway stations place clean toys kids books kitchen wellequipped safe environment highly recommend great place equip kitchen amazing location fantastic fantastic try rebook hope jed home available time travel city wonderful way exceptional location jed olga responsive space clean need lovely cozy perfectly organize street level apartment great location absolutely come stay jed renovation apartment neighborhood grateful immediately feel home great stay place clean extremely comfortable right size family kids love toys books place locate perfect visit nyclean relatively ample manhattan close shops restaurantsthe host responsive strongly recommend excellent location stock kitchen overall comfortable apt home away home great family kids enjoy books toys hosts responsive internet issue flexible check intime,home jed boys absolutely museum amazing fantastic clean need absolutely stay ample home
246,"New York, United States",4,broadway location midtown studio,996,Entire rental unit,4.8,essential spot stay trip nyc location perfect need want eat comfortable clean studio apartment base explore definitely recommend stay henry thanks nice easy access host small space clean job great location great location super clean little place stay visit nyc great central location easy walk subways train stations room bed bugs overall building run like cheap hotel pictures online nt match actual room amenities order see bed bugs crawl ruin stay helpful host room clean quiet easy location henry communicative place walking distance want apartment remarkably clean highly recommend amazing 

## Recommendation system

In [82]:
dfs_03 = df_s02.copy()
dfs_03.fillna('', inplace=True)
dfs_03.head(1)

,address,numberOfGuests,name,pricing/rate/amount,roomType,stars,comments,relevant_words
0,"Jersey City, New Jersey, United States",2,entire cozy unit mins manhattan,99999,Entire rental unit,4.67,great location nice building love parking include stay perfect mention listing location min walk newport path right opposite huge grocery store joes super kind early checkin late checkout excellent place stay central fo travel nyc work travel new jersey stay months apartment place advertise joes excellent host best airbnb stay get great view outside window comfortable convenient living experience settle great nice apartment great locality apartment list definitely rate star airbnb pet hair carpet couch month old bit concerned have allergies look clean place joes approachable easy communicate customer take chance trust listing apartment exceed expectations super clean aesthetically pleasing minimalistic vibe definitely luxury apartment building convenient location right shoprite near waterfront path host responsive questions thanks great stay,building love checkin central advertise advertise great apartment apartment


In [83]:
# TF-IDF
max_words = 10000
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), max_features=max_words)
#tfidf_matrix = vectorizer.fit_transform(dfs_03['comments'] + "" + dfs_03['relevant_words'])
tfidf_matrix = vectorizer.fit_transform(dfs_03['relevant_words'])
tfidf_feature_names = vectorizer.get_feature_names_out()

In [107]:
input_search = str(input())
df_input = pd.DataFrame(np.array([[input_search]]))
pp_input = vectorizer.transform(df_input[0])

cozy


In [109]:
# Computing cosine similarity of user input
cos_similarity_tfidf = map(lambda x: cosine_similarity(pp_input, x), tfidf_matrix)

In [110]:
simi_scores = list(cos_similarity_tfidf)
simi_scores = [(i, score.squeeze(0)[0]) for i, score in enumerate(simi_scores)]
simi_scores = sorted(simi_scores, key=lambda x: x[1], reverse=True)

In [111]:
announces_index =[i[0] for i in simi_scores]

In [112]:
top_n = 5
dfs_03.iloc[announces_index][:top_n]

,address,numberOfGuests,name,pricing/rate/amount,roomType,stars,comments,relevant_words
317,"New York, United States",4,bedrooms baths luxury doorman condo,750,Entire condo,,place awesome darren great host location place unbelievable actual apartment perfect definitely stay darren great host place cozy equip relax welldressed ill,place cozy
488,"New York, United States",6,beautiful br apt heart big apple,521,Entire rental unit,4.25,apartment situate lot foot great beds comfy share room pleasant unfortunately disappoint listing reasons clunk pipe bedroom bathroom show pictures cleanliness issues false street address give security reasons unclean kitchenware miss kitchen cloths wifi issues host responsive problems fix expect quality tidiness apartment feel overpriced ve get great location easy ny show roselyn accommodate alaways sure need choice area fault entirely prefer lower times square great location apartment cozy clean love host roselyn nice helpful happily book tidy lovely airbnb apartment equip need good bed nice bathroom roselyn great host need nice want share know area great stay host superb give clear instructions get apartment space absolutely promise apartment stylish locate fantastic area easy walking distance great cafes shops highly recommend apartment look appoint locate spot explore city regular airbnb traveler rate place nicest best locate real value places stay place clean modern need feel home easy walk sites want plus steps dozens restaurants cafes communication great highly recommend wonderful apartment die wohnung ist sehr zentral gelegen und die kommunkation mit roselyn war unkompliziert und freundlich dinge die wir beanstandet haben wurden sofort behoben danke dafur allerdings konnte die wohnung mehr pflege gebrauchen,issues easy ny accommodate apartment cozy nice apartment place die
277,"New York, United States",2,comfortable entire large br nd floor washdryer,614,Entire rental unit,4.95,excellent placegreat location sure roomy unit giorgio willing work minor concerns neighborhood convenient walk subway busses good restaurants con windows look cable tv flight convenient recommend unit amazing place want local new yorker experience look definitely highly recommend joanna comfortable stay incredible location close restaurants subway central park wifi work perfectly remote work video calls giorgio responsive sure comfortable stay definitely time visit nyc giorgios place month june apartment incredibly quiet noise neighbors clean convenient great location etc comfortable clean authentic ny apartment giorgio super responsive accommodate checkin process extremely smooth apartment entirely expectations recommend want live authentic upper west apartment nice flat nice street close central park restaurants amazing grocery shopone remark living room dark view cosy great place amazing locationthe apartment comfortable stay againgiorgio accommodating sure ready staythanks giorgio mark good location extremely comfortable bed great washer dryer great location cozy place,excellent unit amazing place comfortable stay apartment incredibly apartment nice accommodating bed cozy
462,"New York, United States",2,million dollar view spacious luxurybed apartment,522,Entire rental unit,4.79,best view manhattanvicky awesome host responsive easy work ll beautiful view house neat feel like cleaner bare minimum go reorganize things vacuum spots ocdother home beautiful view spectacular lots places eat great cafe end block pay park worth place stunning views keep cozy enjoy stay book magnificent view nychost super responsive accommodate amazing views great location thank place impeccable breathtaking view proximity restaurants shops tourist sites cozy furnishings necessities regret book place amazing spot amazing host say awesome place unbelievable view wife nt believe eyes step door jawdropping vicky great host communicative friendly easy checkin great definitely stay vicky communicate appreciate responsiveness definitely feel wan